Experiment to use lora to make a lying model. Here we think of Lora as a probe, as it acts in a very similar way - modifying the residual stream.

Then the hope is it will assist at lie detecting and generalize to unseen dataset

- https://github.dev/JD-P/minihf/blob/b54075c34ef88d9550e37fdf709e78e5a68787c4/lora_tune.py
- https://github.com/jonkrohn/NLP-with-LLMs

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

plt.style.use("ggplot")

from typing import Optional, List, Dict, Union
from jaxtyping import Float
from torch import Tensor

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
from einops import rearrange

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, LoftQConfig, IA3Config

import datasets
from datasets import Dataset

from loguru import logger

logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


# # quiet please
torch.set_float32_matmul_precision("medium")
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings(
#     "ignore", ".*sampler has shuffling enabled, it is strongly recommended that.*"
# )
# warnings.filterwarnings("ignore", ".*has been removed as a dependency of.*")


In [ ]:
# load my code
%load_ext autoreload
%autoreload 2

import lightning.pytorch as pl
from src.datasets.dm import DeceptionDataModule
from src.models.pl_lora_ft import AtapterFinetuner

from src.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset, load_preproc_datasets
from src.models.load import load_model
from src.helpers.torch import clear_mem
from src.models.phi.model_phi import PhiForCausalLMWHS


## Parameters


In [ ]:
# params
max_epochs = 2
device = "cuda:0"

cfg = ExtractConfig(
    max_examples=(300, 100),
    
    # model="wassname/phi-1_5-w_hidden_states",
    # batch_size=3,

    # model="wassname/phi-2-w_hidden_states",
    model="Walmart-the-bag/phi-2-uncensored",
    batch_size=1,
    prompt_format="phi",
)


## Load model

In [ ]:
model, tokenizer = load_model(
    cfg.model,
    device=device,
    model_class=PhiForCausalLMWHS,
)


In [ ]:
# TODO I would like to only have biases, but for now lets just try a very small intervention on the last parts of a layer...
peft_config = LoraConfig(
    target_modules=[
        "out_proj",
        "mlp.fc2",
    ],  # only the layers that go directly to the residual
    # bias="lora_only",
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=4,
    lora_dropout=0.0,
)


# peft_config = IA3Config(
#     task_type=TaskType.SEQ_CLS, target_modules=[ "out_proj",
#         "mlp.fc2",], feedforward_modules=["out_proj", "mlp.fc2",]
# )
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


## Load datasets

In [ ]:
N = sum(cfg.max_examples)
ds_tokens = load_preproc_datasets(cfg.datasets,
                      tokenizer,
        N=N,
        seed=cfg.seed,
        num_shots=cfg.num_shots,
        max_length=cfg.max_length,
        prompt_format=cfg.prompt_format,
)
ds_tokens


In [ ]:
ds_tokens2 = load_preproc_datasets(cfg.datasets_oos,
                      tokenizer,
        N=N//2,
        seed=cfg.seed,
        num_shots=cfg.num_shots,
        max_length=cfg.max_length,
        prompt_format=cfg.prompt_format,
)
ds_tokens2


## Train

In [ ]:
dm = DeceptionDataModule(ds_tokens, batch_size=cfg.batch_size)
dm


In [ ]:
dl_train = dm.train_dataloader()
dl_val = dm.val_dataloader()


In [ ]:
b = next(iter(dl_train))
print(b.keys(), b["input_ids"].shape)
c_in = b["input_ids"].shape[1]
c_in


## custom models

In [ ]:
# from src.models.pl_lora_ft import AtapterFinetuner, select_choices

# class AtapterFinetunerLie(AtapterFinetuner):
#     def get_loss(self, batch, out, out_a):
#         """
#         simply train it to lie
#         """

#         log_probs_a = torch.log_softmax(out_a["logits"][:, -1,], -1,)

#         # batch['instructed_to_lie']
#         lie_label = ~batch['label_true']
#         # choice_ids1 = batch['choice_ids'][:, lie_label, 0]
#         choice_ids1 = batch['choice_ids'][:, :, 0][torch.arange(1).long().unsqueeze(1), lie_label.long()].squeeze(1)
#         # choice_ids2 = batch['choice_ids'][:, lie_label, 1]
#         choice_ids2 = batch['choice_ids'][:, :, 1][torch.arange(1).long().unsqueeze(1), lie_label.long()].squeeze(1)
#         # choice_ids = batch['choice_ids'][torch.arange(1).long().unsqueeze(1), lie_label.long()]
#         loss1 = F.nll_loss(log_probs_a, target=choice_ids1)        
#         loss2 = F.nll_loss(log_probs_a, target=choice_ids2)
#         loss = (loss1 + loss2) / 2

#         return loss, None, None


In [ ]:
from src.models.pl_lora_ft import AtapterFinetuner, select_choices

class AtapterFinetunerToldToLie(AtapterFinetuner):
    def get_loss(self, batch, out, out_a):
        """
        simply train it to lie
        """

        log_probs_a = torch.log_softmax(out_a["logits"][:, -1,], -1,)

        
        lie_label = batch['label_true'] ^ batch['instructed_to_lie']
        # choice_ids1 = batch['choice_ids'][:, lie_label, 0]
        choice_ids1 = batch['choice_ids'][:, :, 0][torch.arange(1).long().unsqueeze(1), lie_label.long()].squeeze(1)
        # choice_ids2 = batch['choice_ids'][:, lie_label, 1]
        choice_ids2 = batch['choice_ids'][:, :, 1][torch.arange(1).long().unsqueeze(1), lie_label.long()].squeeze(1)
        # choice_ids = batch['choice_ids'][torch.arange(1).long().unsqueeze(1), lie_label.long()]
        loss1 = F.nll_loss(log_probs_a, target=choice_ids1)        
        loss2 = F.nll_loss(log_probs_a, target=choice_ids2)
        loss = (loss1 + loss2) / 2

        return loss, None, None


In [ ]:
net = AtapterFinetunerToldToLie(
    model, tokenizer, lr=5e-3, weight_decay=1e-3, total_steps=len(dl_train) * max_epochs
)

print(c_in)
# net.model.enable_adapters()


In [ ]:
# # debug
# with torch.no_grad():
#     o = net.training_step(b, None)
# o


In [ ]:
# # debug
# with torch.no_grad():
#     o = net.predict_step(b, None)
# o.keys()


In [ ]:
# we want to init lightning early, so it inits accelerate
trainer1 = pl.Trainer(
    # precision="16-true", # leads to inf loss?
    # precision="16-mixed", # works
    # precision="bf16-mixed",
    gradient_clip_val=20,
    # accelerator="auto",
    devices="1",
    accelerator="gpu",
    # devices=[0],
    accumulate_grad_batches=4,
    max_epochs=max_epochs,
    log_every_n_steps=1,
    enable_model_summary=False,
)


In [ ]:
trainer1.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val);


In [ ]:
checkpoint_path = Path(trainer1.log_dir)/'final'
model.save_pretrained(checkpoint_path)
checkpoint_path


In [ ]:
from src.helpers.lightning import read_metrics_csv

df_histe, df_hist = read_metrics_csv(trainer1.logger.experiment.metrics_file_path)
df_hist[['train/loss_step', 'val/loss_step']].plot(style='.')
df_hist


In [ ]:
df_histe.drop(columns=['step']).plot()


## Generate


In [ ]:
model, tokenizer = model, tokenizer = load_model(
    cfg.model,
    device=device,
    adaptor_path=checkpoint_path,
    dtype=torch.float16, # bfloat can't be pickled
    model_class=PhiForCausalLMWHS,
)
clear_mem()


In [ ]:
# get a row
bi = 4
inputs = ds_tokens.with_format("torch")[bi]

from src.eval.gen import gen


In [ ]:
with model.disable_adapter():
    gen(model, inputs, tokenizer)

gen(model, inputs, tokenizer)


# Test

In [ ]:
from src.eval.helpers import test_intervention_quality2
from src.eval.labels import ds2label_model_obey, ds2label_model_truth


In [ ]:
dm = DeceptionDataModule(ds_tokens, batch_size=cfg.batch_size * 2)
dl_train2 = dm.train_dataloader()
dl_val2 = dm.val_dataloader()
dl_test2 = dm.test_dataloader()


In [ ]:
dl_oos2 = DataLoader(
    ds_tokens2, batch_size=cfg.batch_size * 2, drop_last=False, shuffle=False
)
len(ds_tokens2)


In [ ]:
# rs = trainer1.test(
#     net,
#     dataloaders=[
#         # dl_train2, dl_val2,
#         dl_test2,
#         dl_oos2,
#     ],
# )
# rs = rename(rs, ["train", "val", "test", "oos"])
# rs[0]


# Predict

Here we want to see if we can do a probe on the hidden states to see if it's lying...


### Collect

- see how acc each was for instructions vs truth
- see how a linear probe trained on the diff can do for truth, vs baseline

In [ ]:
model, tokenizer = model, tokenizer = load_model(
    cfg.model,
    device=device,
    adaptor_path=checkpoint_path,
    dtype=torch.float16, # bfloat can't be pickled
    model_class=PhiForCausalLMWHS,
)
clear_mem()


In [27]:
from src.eval.collect import manual_collect2
ds_out, f = manual_collect2(dl_oos2, model, dataset_name="oos")
ds_out


collecting hidden states: 100%|██████████| 100/100 [04:19<00:00,  2.59s/it]


Dataset({
    features: ['end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'label_instructed_base', 'instructed_to_lie_base', 'sys_instr_name_base', 'example_i_base', 'ds_string_base', 'template_name_base', 'correct_truth_telling_base', 'correct_instruction_following_base', 'end_residual_stream_base', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'label_instructed_adapt', 'instructed_to_lie_adapt', 'sys_instr_name_adapt', 'example_i_adapt', 'ds_string_adapt', 'template_name_adapt', 'correct_truth_telling_adapt', 'correct_instruction_following_adapt', 'end_residual_stream_adapt'],
    num_rows: 200
})

### Eval

In [28]:
from src.eval.labels import LABEL_MAPPING


In [29]:
# TODO limit it to ones where it knows
for label_name, label_fn in LABEL_MAPPING.items():
    # fit probe
    print('='*80)
    print('making intervention with', label_name, 'hidden states')
    test_intervention_quality2(ds_out, label_fn, tokenizer)


making intervention with label_model_truth hidden states
primary metric: predictive power (of logistic regression on top of intervened hidden states)
predictive power? False [i] = baseline: 0.586 > 0.549 roc_auc
predictive power? False [i-b] = baseline: 0.586 > 0.526 roc_auc
predictive power? False [diff]  = baseline: 0.565 > 0.520 roc_auc
predictive power? False [diff(i-b)] = baseline: 0.565 > 0.526 roc_auc
          baseline  intervene
coverage  0.815331   0.774906
ans       0.671902   0.558883
making intervention with label_model_obey hidden states
primary metric: predictive power (of logistic regression on top of intervened hidden states)
predictive power? False [i] = baseline: 0.580 > 0.590 roc_auc
predictive power? False [i-b] = baseline: 0.580 > 0.550 roc_auc
predictive power? False [diff]  = baseline: 0.570 > 0.590 roc_auc
predictive power? False [diff(i-b)] = baseline: 0.570 > 0.560 roc_auc
          baseline  intervene
coverage  0.815331   0.774906
ans       0.671902   0.5588

### Check dataset of outputs

In [30]:
from src.eval.ds import qc_ds, ds2df, qc_dsdf


In [31]:
df = ds2df(ds_out)
df


,binary_ans_base,label_true_base,label_instructed_base,instructed_to_lie_base,sys_instr_name_base,example_i_base,ds_string_base,template_name_base,correct_truth_telling_base,correct_instruction_following_base,...,sys_instr_name_adapt,example_i_adapt,ds_string_adapt,template_name_adapt,correct_truth_telling_adapt,correct_instruction_following_adapt,choice_probs_adapt,ans_adapt,choice_probs_base,ans_base
0,0.209368,False,True,True,sphinx,192,glue:qnli,have all you need,0.790632,0.209368,...,sphinx,192,glue:qnli,have all you need,0.758412,0.241588,0.840402,False,0.975285,False
1,0.002433,False,False,False,truth,192,glue:qnli,based only on,0.997567,0.997567,...,truth,192,glue:qnli,based only on,0.959976,0.959976,0.757396,False,0.969436,False
2,0.396011,True,False,True,lie_for_charity,73,glue:qnli,imply,0.396011,0.603989,...,lie_for_charity,73,glue:qnli,imply,0.453921,0.546079,0.833045,False,0.991657,False
3,0.449343,True,True,False,truth,73,glue:qnli,have all you need,0.449343,0.449343,...,truth,73,glue:qnli,have all you need,0.461182,0.461182,0.916895,False,0.993043,False
4,0.987551,True,False,True,guard,233,glue:qnli,based only on,0.987551,0.012449,...,guard,233,glue:qnli,based only on,0.848788,0.151212,0.264317,True,0.391617,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.872044,False,False,False,truth,298,glue:qnli,based only on,0.127956,0.127956,...,truth,298,glue:qnli,based only on,0.323592,0.323592,0.784727,True,0.076331,True
196,0.870453,True,False,True,guard,91,glue:qnli,have all you need,0.870453,0.129547,...,guard,91,glue:qnli,have all you need,0.680157,0.319843,0.928511,True,0.993703,True
197,0.789229,True,True,False,truth,91,glue:qnli,imply,0.789229,0.789229,...,truth,91,glue:qnli,imply,0.750310,0.750310,0.347062,True,0.011330,True
198,0.705778,True,False,True,lie_for_charity,243,glue:qnli,have all you need,0.705778,0.294222,...,lie_for_charity,243,glue:qnli,have all you need,0.546683,0.453317,0.941037,True,0.994001,True


In [32]:
choice_probs_adapt


NameError: name 'choice_probs_adapt' is not defined

In [ ]:
# TODO one for base, one for adapter
# TODO is acc and lie_acc the same... so it's ignoring the examples and system instrucitons... maybe I need a instruction tuned one?
qc_ds(ds_out)


In [ ]:
print('acc by dataset and template name')
df1 = ds2df(ds_out)
df_b = df1.rename(columns=lambda x: x.replace('_base', '')).copy()
df_a = df1.rename(columns=lambda x: x.replace('_adapt', '')).copy()
for ds_string, ddf in df_b.groupby(['ds_string', 'template_name']):
    print(ds_string)
    qc_dsdf(ddf)
    



In [ ]:
print('acc by dataset and sys_instr_name')
df = ds2df(ds_out.with_format('numpy')).rename(columns=lambda x: x.replace('_base', ''))
df['ans'] = df['binary_ans'] >0.5
df['label_instructed'] = df['label_true'] ^ df['instructed_to_lie']
for ds_string, ddf in df.groupby(['ds_string','sys_instr_name']):
    print(ds_string)
    qc_dsdf(ddf)
